In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/facial_expression_detection/research'

In [2]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/facial_expression_detection'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [11]:
from src.detmood.constant import *
from src.detmood.utils.main_utils import create_directories, read_yaml

class ConfigurationMananger:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
    
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )
        
        return data_ingestion_config

In [27]:
import os
from src.detmood.utils.main_utils import get_size, create_directories
from src.detmood import logger
import zipfile
import subprocess

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_dataset(self):
        if not os.path.exists(self.config.local_data_file):
            info = subprocess.run(
                f"wget '{self.config.source_URL}' -O {self.config.local_data_file}",
                shell=True
            )
            logger.info(f"Dataset downloaded with folowing info: \n{info}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.root_dir)
        logger.info("Zip file extacted!")
        
        os.remove(self.config.local_data_file)
        logger.info("Zip file removed!")

In [28]:
try:
    config = ConfigurationMananger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_dataset()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-10-26 00:28:30,404: INFO: main_utils: created directory at: artifacts]
[2024-10-26 00:28:30,413: INFO: main_utils: created directory at: artifacts/data_ingestion]


--2024-10-26 00:28:30--  https://drive.usercontent.google.com/download?id=184K8sLVcL_fq76pA2o8Bg4BthNLcwUel&export=download&authuser=0&confirm=t
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.20.1, 2a00:1450:400d:805::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.20.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27388816 (26M) [application/octet-stream]
Saving to: ‘artifacts/data_ingestion/labels_and_dataset.zip’

     0K .......... .......... .......... .......... ..........  0% 1,19M 22s
    50K .......... .......... .......... .......... ..........  0% 2,44M 16s
   100K .......... .......... .......... .......... ..........  0% 4,99M 13s
   150K .......... .......... .......... .......... ..........  0% 4,68M 11s
   200K .......... .......... .......... .......... ..........  0% 7,14M 9s
   250K .......... .......... .......... .......... ..........  1% 4,43M 9s
   300K ..........

[2024-10-26 00:28:33,403: INFO: 4134096572: Dataset downloaded with folowing info: 
CompletedProcess(args="wget 'https://drive.usercontent.google.com/download?id=184K8sLVcL_fq76pA2o8Bg4BthNLcwUel&export=download&authuser=0&confirm=t' -O artifacts/data_ingestion/labels_and_dataset.zip", returncode=0)]


....... .......... .......... 90% 29,1M 0s
 24250K .......... .......... .......... .......... .......... 90% 36,8M 0s
 24300K .......... .......... .......... .......... .......... 91% 38,1M 0s
 24350K .......... .......... .......... .......... .......... 91% 27,5M 0s
 24400K .......... .......... .......... .......... .......... 91% 35,4M 0s
 24450K .......... .......... .......... .......... .......... 91% 41,5M 0s
 24500K .......... .......... .......... .......... .......... 91% 32,7M 0s
 24550K .......... .......... .......... .......... .......... 91% 33,1M 0s
 24600K .......... .......... .......... .......... .......... 92% 36,3M 0s
 24650K .......... .......... .......... .......... .......... 92% 36,5M 0s
 24700K .......... .......... .......... .......... .......... 92% 39,0M 0s
 24750K .......... .......... .......... .......... .......... 92% 27,6M 0s
 24800K .......... .......... .......... .......... .......... 92% 34,7M 0s
 24850K .......... .......... .......... ....

[2024-10-26 00:28:34,756: INFO: 4134096572: Zip file extacted!]
[2024-10-26 00:28:34,761: INFO: 4134096572: Zip file removed!]
